# Saving and Loading Gluon Models

In reality you almost always save the models you train into files. There is a number of reasons to do this. For example,
1. You might want to do inference on a machine that is different from the one where the model was trained.
2. It is possible the model's performance on validation set decreased towards the end of the training for reasons like overfitting. If you saved your model parameters after every epoch, at the end you can decide to use the model that performs best on the vaidation set.

So, we need ways to save models to file and restore them back from file. In this tutorials we will learn ways to save and load Gluon models.

Let's start by importing the modules we'll need.

In [ ]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon

import numpy as np

## Build and train a simple model

We need a trained model before we can save it to a file. So let's go ahead and build a very simple convolutional network and train it on MNIST data.

In [ ]:
# Use GPU if one exists, else use CPU
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

# MNIST images are 28x28. Total pixels in input layer is 28x28 = 784
num_inputs = 784
# Clasify the images into one of the 10 digits
num_outputs = 10
# 64 images in a batch
batch_size = 64

# Helper to preprocess data for training
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

# Load the training data
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=transform), 
                                   batch_size, shuffle=True)

# A helper method to train a given model using MNIST data
def train_model(model):

    # Initialize the parameters with Xavier initializer
    net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
    # Use cross entropy loss
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    # Use Adam optimizer
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': .001})

    # Train for one epoch
    for epoch in range(1):
        # Iterate through the images and labels in the training data
        for batch_num, (data, label) in enumerate(train_data):
            # get the images and labels
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            # Ask autograd to record the forward pass
            with autograd.record():
                # Run the forward pass
                output = model(data)
                # Compute the loss
                loss = softmax_cross_entropy(output, label)
            # Compute gradients
            loss.backward()
            # Update parameters
            trainer.step(data.shape[0])

            # Print loss once in a while
            if batch_num % 50 == 0:
                curr_loss = nd.mean(loss).asscalar()
                print("Epoch: %d; Batch %d; Loss %f" % (epoch, batch_num, curr_loss))

# Build a simple convolutional network
def build_lenet(net):    
    with net.name_scope():
        # First convolution
        net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
        net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        # Second convolution
        net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
        net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        # Flatten the output before the fully connected layers
        net.add(gluon.nn.Flatten())
        # First fully connected layers with 512 neurons
        net.add(gluon.nn.Dense(512, activation="relu"))
        # Second fully connected layer with as many neurons as the number of classes
        net.add(gluon.nn.Dense(num_outputs))
        
        return net

net = gluon.nn.Sequential()
net = build_lenet(net)
train_model(net)

## Saving model parameters to file

Okay, we now have a model (`net`) that we can save to a file. Let's save the parameters of this model to a file.

In [ ]:
file_name = "net.params"
net.save_params(file_name)

That's it! We have successfully saved the parameters of the model into a file.

## Loading model parameters from file

Let's now create a network with the parameters we saved into the file.

In [ ]:
new_net = build_lenet()
new_net.load_params(file_name, ctx=ctx)

Let's test the model we just loaded from file.

In [ ]:
import matplotlib.pyplot as plt

def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)

# Load ten random images from the test dataset
sample_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                              10, shuffle=True)

for data, label in sample_data:

    # Display the images
    img = nd.transpose(data, (1,0,2,3))
    img = nd.reshape(img, (28,10*28,1))
    imtiles = nd.tile(img, (1,1,3))
    plt.imshow(imtiles.asnumpy())
    plt.show()

    # Display the predictions
    data = nd.transpose(data, (0, 3, 1, 2))
    out = new_net(data.as_in_context(ctx))
    predictions = nd.argmax(out, axis=1)
    print('Model predictions: ', predictions.asnumpy())

    break

## Saving model architecture and weights to file

Hybrid models can be serialized as JSON files using the `export` function. Once serialized, these models can be loaded from other language bindings like C++ or Scala for faster inference or inferences in different environments.

Note that the network we created above is not a Hybrid network and therefore cannot be serializes into a JSON file. So, let's create a Hybrid version of the same network and train it.

In [ ]:
net = gluon.nn.HybridSequential()
net = build_lenet(net)
net.hybridize()
train_model(net)

In [ ]:
net.export("lenet", epoch=1)

That's it! `export` in this case creates `lenet-symbol.json` and `lenet-0001.params` in the current directory.

## Loading model architecture and weights from a different frontend

Primary reason to serialize model architecture into a JSON file is to load it from a different frontend like C, C++ or Scala. Here is a couple of examples:
1. [Loading serialized Hybrid networks from C](https://github.com/apache/incubator-mxnet/blob/master/example/image-classification/predict-cpp/image-classification-predict.cc)
2. [Loading serialized Hybrid networks from Scala](https://github.com/apache/incubator-mxnet/blob/master/scala-package/infer/src/main/scala/org/apache/mxnet/infer/ImageClassifier.scala)

## Loading model architecture from Python

Serialized Hybrid networks (saved as .JSON and .params file) can be loaded and used inside Python frontend using `mx.model.load_checkpoint` and `gluon.nn.SymbolBlock`. To demonstrate that, let's load the network we serialized above.